# Apply trace metal concentrations to river classification

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean

%matplotlib inline

## River categories:

1. Glaciers
2. Continental
3. Other

Load river classification file: river_class_201807.nc

In [21]:
rclass = nc.Dataset('/ocean/brogalla/GEOTRACES/data/rivers/river_class_201807.nc','r')
river_class = rclass.variables['rclass']
river_class = np.array(river_class)

### Apply trace metal concentrations

Notebook to find these concentratons: /analysis-brogalla/forcing/rivers/find_characteristic_tracer_concentrations.ipynb

In [22]:
pMn_rivers = np.zeros(river_class.shape)
dMn_rivers = np.zeros(river_class.shape)
pPb_rivers = np.zeros(river_class.shape)
dPb_rivers = np.zeros(river_class.shape)

Manganese:

In [24]:
pMn_rivers[river_class==1.0] = 151*1000 # glaciers
pMn_rivers[river_class==2.0] = 13  # continental
pMn_rivers[river_class==3.0] = 2   # other

In [25]:
dMn_rivers[river_class==1.0] = 164*1000 # glaciers
dMn_rivers[river_class==2.0] = 30  # continental
dMn_rivers[river_class==3.0] = 2   # other

Lead:

In [26]:
pPb_rivers[river_class==1.0] = 5 # glaciers
pPb_rivers[river_class==2.0] = 1  # continental
pPb_rivers[river_class==3.0] = 0.2   # other

In [27]:
dPb_rivers[river_class==1.0] = 35 # glaciers
dPb_rivers[river_class==2.0] = 43  # continental
dPb_rivers[river_class==3.0] = 2   # other

#### Convert units to mol/L:

In [28]:
dMn_rivers = dMn_rivers*10**(-9)      # nM --> 10^-3 mol/L
pMn_rivers = pMn_rivers*(1/54.938049)*10**(-6) # micro g/L --> 1.82*10^-2 mol/L

In [29]:
print(np.amax(dMn_rivers))
print(np.amax(pMn_rivers))

0.000164
0.0027485504627221113


In [30]:
dPb_rivers = dPb_rivers*10**(-12)           # pM --> 10^-3 mol/L
pPb_rivers = pPb_rivers*(1/207.2)*10**(-6) # micro g/L --> 1.82*10^-2 mol/L

In [31]:
print(np.amax(dPb_rivers))
print(np.amax(pPb_rivers))

4.3e-11
2.413127413127413e-08


# Write to NetCDF file

In [32]:
c = nc.Dataset('/ocean/brogalla/GEOTRACES/data/runoff/runoff_core_monthly.nc','r')
lon_rf = np.array(c.variables['nav_lon'])
lat_rf = np.array(c.variables['nav_lat'])

In [33]:
ncd = nc.Dataset('/ocean/brogalla/GEOTRACES/data/rivers/river_input-test-case.nc', 'w', zlib=True)
# nc_tools.init_dataset_attrs(
#     ncd,
#     title='Tracer input from rivers',
#     notebook_name='rivers',
#     nc_filepath='./river_input.nc',
#     comment='Input of tracer concentration from rivers')

ncd.createDimension('x',len(c.dimensions['x']))
ncd.createDimension('y',len(c.dimensions['y']))

<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 2400

In [34]:
# variables
nav_lat = ncd.createVariable('nav_lat', 'float32', ('y','x'))
nav_lat.long_name = 'Latitude'
nav_lat.units = 'degrees_north'
nav_lat[:] = lat_rf

nav_lon = ncd.createVariable('nav_lon', 'float32', ('y','x'))
nav_lon.long_name = 'Longitude'
nav_lon.units = 'degrees_east'
nav_lon[:] = lon_rf

pmn_rivers = ncd.createVariable('pmn_rivers', 'float32', ('y','x'))
pmn_rivers.units = 'mol/L'
pmn_rivers.long_name = 'Oxidised Mn concentration'  
pmn_rivers.coordinates = 'nav_lon nav_lat'
pmn_rivers[:] = pMn_rivers

dmn_rivers = ncd.createVariable('dmn_rivers', 'float32', ('y','x'))
dmn_rivers.units = 'mol/L'
dmn_rivers.long_name = 'Dissolved Mn concentration'  
dmn_rivers.coordinates = 'nav_lon nav_lat'
dmn_rivers[:] = dMn_rivers

ppb_rivers = ncd.createVariable('ppb_rivers', 'float32', ('y','x'))
ppb_rivers.units = 'mol/L'
ppb_rivers.long_name = 'Particulate Pb concentration'  
ppb_rivers.coordinates = 'nav_lon nav_lat'
ppb_rivers[:] = pPb_rivers

dpb_rivers = ncd.createVariable('dpb_rivers', 'float32', ('y','x'))
dpb_rivers.units = 'mol/L'
dpb_rivers.long_name = 'Dissolved Pb concentration'  
dpb_rivers.coordinates = 'nav_lon nav_lat'
dpb_rivers[:] = dPb_rivers

In [35]:
ncd.close()

In [37]:
c = nc.Dataset('/ocean/brogalla/GEOTRACES/data/rivers/river_input-base-case.nc','r')
print(np.amax(c.variables['dmn_rivers']))
print(np.amax(c.variables['pmn_rivers']))
print(np.amax(c.variables['dpb_rivers']))
print(np.amax(c.variables['ppb_rivers']))

1.64e-07
2.7485505e-06
4.3e-11
2.4131275e-08


Glacial river trace metal concentrations enhanced by 50\%:

In [17]:
c = nc.Dataset('/ocean/brogalla/GEOTRACES/data/rivers/river_input-glacier-enhanced.nc','r')
print(np.amax(c.variables['dmn_rivers']))
print(np.amax(c.variables['pmn_rivers']))
print(np.amax(c.variables['dpb_rivers']))
print(np.amax(c.variables['ppb_rivers']))

2.46e-07
4.122826e-06
4.3e-11
2.4131275e-08


Continental river trace metal concentration decreased by 50%:

In [18]:
c = nc.Dataset('/ocean/brogalla/GEOTRACES/data/rivers/river_input-decrease-case.nc','r')
print(np.amax(c.variables['dmn_rivers']))
print(np.amax(c.variables['pmn_rivers']))
print(np.amax(c.variables['dpb_rivers']))
print(np.amax(c.variables['ppb_rivers']))

1.64e-07
2.7485505e-06
4.3e-11
2.4131275e-08


Continental river trace metal concentration increased by 50\%:

In [19]:
c = nc.Dataset('/ocean/brogalla/GEOTRACES/data/rivers/river_input-increase-case.nc','r')
print(np.amax(c.variables['dmn_rivers']))
print(np.amax(c.variables['pmn_rivers']))
print(np.amax(c.variables['dpb_rivers']))
print(np.amax(c.variables['ppb_rivers']))

1.64e-07
2.7485505e-06
4.3e-11
2.4131275e-08
